# pyarrow 방식 성능 테스트  
- cache 누적 처리 시 성능 저하 문제 회피  
- 메모리 적재 형태 데이터를 spark로 전달하고 처리 후 다시 pandas 형태로 처리하는 테스트   
- hdfs data는 미리 접근 가능한 위치로 옮기고 처리한다 가정...  


In [2]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
# import findspark
import time 
import numpy as np 
import pandas as pd
import pyarrow 

In [50]:
# !pip install findspark

In [2]:
# import findspark
# findspark.init()

In [3]:
# # sys.version
# # pd.__version__
# import pyspark
# pyspark.__version__
# # !pip install pandas==1.3.0
# pyarrow.__version__

In [3]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = 'RDD-Pandas'
DB_NAME = 'inven'

# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = 'inven/table-set-6m-20-1000'
file_format = 'parquet' 


In [4]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.rpc.message.maxSize', '2046')\
    .config('spark.driver.maxResultSize', '3000000000')\
    .config('spark.sql.execution.arrow.enabled', 'true')\
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true')\
    .config('spark.sql.execution.arrow.pyspark.fallback.enabled', 'true')\
    .config('spark.driver.cores', '1').config('spark.driver.memory', '7g')\
    .config('spark.num.executors', '3')\
    .config('spark.executor.cores', '1').config('spark.executor.memory', '7g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar').getOrCreate()
    #     .config('spark.sql.execution.arrow.enabled', 'true')\
    # spark.rpc.message.maxSize  240007497 
    sc = spark.sparkContext
    sc
    return spark

In [5]:
%%time
spark = spark_creation()
spark

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


CPU times: user 32.2 ms, sys: 19.7 ms, total: 51.9 ms
Wall time: 20.5 s


In [7]:
# !pip install pyarrow==7.0.0

# 파일 직접 읽기  
parq로 저장된 데이터를 직접 읽어서 pandas & arrow로 처리하기  
hdfs 경로에 직접 접근 못하므로, hdfs 에서 fs로 파일 미리 옯겨 놓는다고 가정  
> pdf 1천만 건을 직접 spark df 변환시에는 mem 부족으로 에러 발생.  
> 나눠서는 변환 가능  
parition key 지정해서 400만 건 이하로 나누고, 읽을 때나 쓸 때 나눠서 처리하면 사용 가능할 듯   

In [5]:
%%time
# df = pyarrow.parquet.read_table(source="out_parq").to_pandas()
# df
import pyarrow.parquet as pq
adf = pq.read_table("parq")
print(adf.shape)
pdf = adf.to_pandas()
pdf.head()

(10000000, 20)
CPU times: user 3.23 s, sys: 1.85 s, total: 5.08 s
Wall time: 3.87 s


,setop,stype,inv_rate_01,inv_val_01,inv_req_01,inv_rate_02,inv_val_02,inv_req_02,inv_rate_03,inv_val_03,inv_req_03,inv_rate_04,inv_val_04,inv_req_04,inv_rate_05,inv_val_05,inv_req_05,inv_rate_06,inv_val_06,inv_req_06
0,ST_B_0499136,ST_B,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0
1,ST_B_0499137,ST_B,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0
2,ST_B_0499138,ST_B,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0
3,ST_B_0499139,ST_B,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0
4,ST_B_0499140,ST_B,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0,0.00001,1000,0


In [6]:
%%time
# 300 ~ 400 만 : 8 초  
# pdf = adf.to_pandas()
# adf = pq.read_table("parq")
mini_pdf = pdf.iloc[2000000:5000000,:]
sdf = spark.createDataFrame(mini_pdf)
sdf.show(1)

+------------+-----+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|       setop|stype|inv_rate_01|inv_val_01|inv_req_01|inv_rate_02|inv_val_02|inv_req_02|inv_rate_03|inv_val_03|inv_req_03|inv_rate_04|inv_val_04|inv_req_04|inv_rate_05|inv_val_05|inv_req_05|inv_rate_06|inv_val_06|inv_req_06|
+------------+-----+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|ST_G_0997120| ST_G|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|
+------------+-----+-----------+----------+----------+-----------+----------+----------+-----------+

# sdf 를 pandas 로 전환  
간단한 pandas dataframe을 만들어서 spark dataframe 으로 변환해 본다.  
pyarrow 버전에 따라 실패할 가능성이 있기 때문에, 간단한 코드로 테스트.  
3.6.9 , pyarrow 6에서는 에러 발생 확인  


In [6]:
# long 컬럼 제외하고 pd로 만들어서 다시 테스트  
pdf = pd.DataFrame()
pdf["setop"] = ["A", "B", "C"]
pdf["val"] = [1, 2, 3]
pdf

,setop,val
0,A,1
1,B,2
2,C,3


In [7]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType, IntegerType
columns = [
    StructField("setop", StringType())
    , StructField("val", IntegerType())]
sample_schema = StructType(columns)

sdf = spark.createDataFrame(pdf, sample_schema)
sdf.show()

+-----+---+
|setop|val|
+-----+---+
|    A|  1|
|    B|  2|
|    C|  3|
+-----+---+



In [8]:
sdf.printSchema()
sdf.show(truncate=False)
# 아래 명령은 에러 발생한다. spark 3.1.2 arrow 6.0.1  
# spark 3.2.1 arrow 6.0.1 에서는 정상 작동  
sdf.toPandas()

root
 |-- setop: string (nullable = true)
 |-- val: integer (nullable = true)

+-----+---+
|setop|val|
+-----+---+
|A    |1  |
|B    |2  |
|C    |3  |
+-----+---+



,setop,val
0,A,1
1,B,2
2,C,3


대용량 데이터에 대한 전환 테스트  

In [14]:
%%time
# spark.rpc.message.maxSize , maxResultSize 늘려도 400만 건 이상에서는 커럽트 발생  
# 400만 23 초  
tbl_setop_name = 'inven/table-set-6m-20-1000'
file_format = 'parquet'

sdf = spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView("tmp")
sdf_tmp = spark.sql("select * from tmp limit 3000000")
# sdf_tmp.rdd.persist()

CPU times: user 5.66 ms, sys: 0 ns, total: 5.66 ms
Wall time: 327 ms


MapPartitionsRDD[47] at javaToPython at NativeMethodAccessorImpl.java:0

In [ ]:
%%time
pdf = sdf_tmp.toPandas()
print(pdf.shape) 
pdf.head()

In [1]:
import gc
del pdf, sdf_tmp, sdf
gc.collect()

NameError: name 'pdf' is not defined

# 직접 전환시 에러 발생해서 우회책  
- spark data를 hive에 파일 기록  
- arrow로 hdfs parq 읽어들이기  
- adf 를 pdf 로 전환  

---  
## arrow로 hdfs parq 읽기
또는 sdf를 adf로 전환  

In [28]:
%%time
#from pyarrow import fs
#fs.HadoopFileSystem("hdfs://namenode:8020?user=hdfsuser")
# hdfs = fs.HadoopFileSystem(host="default")
# hdfs
# hdfs.create_dir("this_is_where")
# print(hdfs.get_file_info('this_is_where'))
# # fs = pyarrow.fs.HadoopFileSystem("hdfs:///")
# df = fs.read_parquet('inven/table-set-6m-20-1000') #, **other_options)
# print('file : ' + str(df.shape))
# df.head()
tbl_setop_name = 'inven/table-set-6m-20-1000'
file_format = 'parquet'

sdf = spark.read.format(file_format).load(tbl_setop_name)

CPU times: user 3.93 ms, sys: 0 ns, total: 3.93 ms
Wall time: 1.49 s


In [11]:
# %%time
# import pyarrow

# # hdfs 연결 테스트  : RPC 연결 에러 발생 상태 !!!!!!!! 
# # rpc port 9000 을 spark edge(master)와 열고 다시 테스트  
# # spark write 시에는 어떻게 hdfs로 넘어가는 지 작동 과정 확인할 것 !!!  
# client = pyarrow.hdfs.HadoopFileSystem("hdfs://namenode:9000")
# client.ls("/user/root")

In [5]:
# help(client)

In [2]:
spark.stop()

NameError: name 'spark' is not defined